In [1]:
!pip install jsonlines
import jsonlines
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
import numpy as np


In [4]:
url = 'https://raw.githubusercontent.com/AmbujaBudakoti27/Sarcasm-Detection/main/sarcasm_detection_shared_task_twitter_training.jsonl'
twitter_train=pd.read_json(url,lines=True)

In [ ]:
twitter_train.head()

,label,response,context
0,SARCASM,@USER @USER @USER I don't get this .. obviousl...,[A minor child deserves privacy and should be ...
1,SARCASM,@USER @USER trying to protest about . Talking ...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,@USER @USER @USER He makes an insane about of ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,@USER @USER Meanwhile Trump won't even release...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,@USER @USER Pretty Sure the Anti-Lincoln Crowd...,[Man ... y ’ all gone “ both sides ” the apoca...


In [5]:
twitter_train['response']=twitter_train['response'].str.replace('@USER', "") 
twitter_train['response']=twitter_train['response'].str.replace('\d+', '')
twitter_train['response']=twitter_train['response'].str.lower()
twitter_train['response']=twitter_train['response'].str.replace('[^\w\s]','')

In [6]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,[A minor child deserves privacy and should be ...
1,SARCASM,trying to protest about talking about him a...,[@USER @USER Why is he a loser ? He's just a P...
2,SARCASM,he makes an insane about of money from the ...,[Donald J . Trump is guilty as charged . The e...
3,SARCASM,meanwhile trump wont even release his sat sc...,[Jamie Raskin tanked Doug Collins . Collins lo...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,[Man ... y ’ all gone “ both sides ” the apoca...


In [7]:
twitter_train['context']=twitter_train['context'].apply(lambda x: ','.join(map(str, x)))
twitter_train['context']=twitter_train['context'].str.replace('@USER', "") 
twitter_train['context']=twitter_train['context'].str.lower()
twitter_train['context']=twitter_train['context'].str.replace('[^\w\s]','')
twitter_train['context']=twitter_train['context'].str.replace('\d+', '')

In [8]:
twitter_train.head()

,label,response,context
0,SARCASM,i dont get this obviously you do care or y...,a minor child deserves privacy and should be k...
1,SARCASM,trying to protest about talking about him a...,why is he a loser hes just a press secretar...
2,SARCASM,he makes an insane about of money from the ...,donald j trump is guilty as charged the evid...
3,SARCASM,meanwhile trump wont even release his sat sc...,jamie raskin tanked doug collins collins look...
4,SARCASM,pretty sure the antilincoln crowd claimed th...,man y all gone both sides the apocalypse o...


In [9]:
twitter_train.tail()

,label,response,context
4995,NOT_SARCASM,you dont i have purchased a lot on amazon c...,apologies for the inconvenience you faced wit...
4996,NOT_SARCASM,emotions you say never knew that i think i ...,idk tho i think i m hungry but that s de...
4997,NOT_SARCASM,you are so right yes silence is not priv...,peace to you and two countries certainly s...
4998,NOT_SARCASM,another lazy delusional voter who takes the...,bernie sanders told elizabeth warren in privat...
4999,NOT_SARCASM,i hope you know no news outlet from nigeria ...,pdp protest brainstorming session deji we nee...


In [10]:
from collections import Counter
Counter(twitter_train['label'])

Counter({'NOT_SARCASM': 2500, 'SARCASM': 2500})

In [11]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
corpus = []

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

for i in range(0, len(twitter_train)):
  m = re.sub('[^a-zA-Z]', ' ', twitter_train["response"][i])
  m = m.lower()
  m = m.split()
  m = [ps.stem(word) for word in m if not word in stopwords.words('english')]
  m =' '.join(m)
  corpus.append(m)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 2500)
X = cv.fit_transform(corpus).toarray()

In [13]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [14]:
y = pd.get_dummies(twitter_train['label'])
y = y.iloc[:,1].values
y

array([1, 1, 1, ..., 0, 0, 0], dtype=uint8)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
from sklearn.naive_bayes import MultinomialNB
sarcasm_detect_model_v1 = MultinomialNB().fit(X_train, y_train)
y_pred = sarcasm_detect_model_v1.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_v1 = accuracy_score(y_test, y_pred)

from sklearn.metrics import confusion_matrix
conf_v1 = confusion_matrix(y_test, y_pred)

In [49]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred)

0.7164480322906155

In [18]:
print("accuracy: ", accuracy_v1, "\n")
print(conf_v1)

accuracy:  0.719 

[[364 155]
 [126 355]]


In [19]:
from sklearn.svm import SVC

model_linear_kernal = SVC(kernel='linear')
model_linear_kernal.fit(X_train, y_train)
model_linear_kernal.score(X_test, y_test)

0.716

In [20]:
model_sigmoid_kernal = SVC(kernel='sigmoid')
model_sigmoid_kernal.fit(X_train, y_train)
model_sigmoid_kernal.score(X_test, y_test)

0.695

In [31]:
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(n_estimators=30)
modelRF.fit(X_train, y_train)
modelRF.score(X_test, y_test)

0.707

In [32]:
from sklearn.ensemble import RandomForestClassifier
modelRF2 = RandomForestClassifier(n_estimators=10)
modelRF2.fit(X_train, y_train)
modelRF2.score(X_test, y_test)

0.673

In [33]:
from sklearn.ensemble import RandomForestClassifier
modelRF3 = RandomForestClassifier(n_estimators=40)
modelRF3.fit(X_train, y_train)
modelRF3.score(X_test, y_test)

0.706

In [36]:
from sklearn.ensemble import RandomForestClassifier
modelRF4 = RandomForestClassifier(n_estimators=60)
modelRF4.fit(X_train, y_train)
modelRF4.score(X_test, y_test)

0.721

In [37]:
from sklearn.ensemble import RandomForestClassifier
modelRF5 = RandomForestClassifier(n_estimators=80)
modelRF5.fit(X_train, y_train)
modelRF5.score(X_test, y_test)

0.725

In [41]:
from sklearn.ensemble import RandomForestClassifier
modelRF6 = RandomForestClassifier(n_estimators=100)
modelRF6.fit(X_train, y_train)
modelRF6.score(X_test, y_test)

0.711

In [40]:
from sklearn.ensemble import RandomForestClassifier
modelRF7 = RandomForestClassifier(n_estimators=120)
modelRF7.fit(X_train, y_train)
modelRF7.score(X_test, y_test)

0.717

In [42]:
from sklearn.ensemble import AdaBoostClassifier
modelAD = AdaBoostClassifier(n_estimators=20)
modelAD.fit(X_train, y_train)
modelAD.score(X_test, y_test)

0.648

In [43]:
from sklearn.ensemble import AdaBoostClassifier
modelAD2 = AdaBoostClassifier(n_estimators=40)
modelAD2.fit(X_train, y_train)
modelAD2.score(X_test, y_test)

0.674

In [44]:
from sklearn.ensemble import AdaBoostClassifier
modelAD3 = AdaBoostClassifier(n_estimators=80)
modelAD3.fit(X_train, y_train)
modelAD3.score(X_test, y_test)

0.685

In [47]:
from sklearn.ensemble import AdaBoostClassifier
modelAD4 = AdaBoostClassifier(n_estimators=100)
modelAD4.fit(X_train, y_train)
modelAD4.score(X_test, y_test)

0.687